# Gaussian Bayesian Classifier

### Mastery Checkpoint 1  
CSC 466, Winter 2022  
Samay Nathani

In [44]:
from pathlib import Path
home = str(Path.home())

In [45]:
%load_ext autoreload
%autoreload 2

import Lab2_helper

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Import our dataset
I chose the [Uber Fares Dataset from Kaggle](https://www.kaggle.com/yasserh/uber-fares-dataset/version/1)

In [57]:
import pandas as pd
import numpy as np
import math
raw_uber_df = pd.read_csv(
    f"uber.csv"
)

Do some preprocessing like Lab 2

In [126]:
features = ['pickup_longitude','pickup_latitude','dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'fare_amount']
raw_uber_df = raw_uber_df.loc[:,features]
display(raw_uber_df.head())

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,fare_amount
0,-73.999817,40.738354,-73.999512,40.723217,1,7.5
1,-73.994355,40.728225,-73.994710,40.750325,1,7.7
2,-74.005043,40.740770,-73.962565,40.772647,1,12.9
3,-73.976124,40.790844,-73.965316,40.803349,3,5.3
4,-73.925023,40.744085,-73.973082,40.761247,5,16.0


Creating a function to find the distance between the drop-off and pick-up locations, credit goes to [Geeks4Geeks]()

In [127]:
def distance(lat1, lat2, lon1, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = math.radians(lon1)
    lon2 = math.radians(lon2)
    lat1 = math.radians(lat1)
    lat2 = math.radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
 
    c = 2 * math.asin(math.sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the result
    return(c * r)

Do some preprocessing to get the distance, passenger count, and fare of an uber

In [128]:
raw_uber_df.dropna(inplace=True)
raw_uber_df['distance'] = raw_uber_df.apply(lambda x: distance(x.loc['pickup_latitude'], x.loc['dropoff_latitude'], x.loc['pickup_longitude'], x.loc['dropoff_longitude']), axis=1)
final_features = ['distance', 'passenger_count', 'fare_amount']
uber_df = raw_uber_df.loc[:, final_features]
uber_df.head()

,distance,passenger_count,fare_amount
0,1.683323,1,7.5
1,2.457590,1,7.7
2,5.036377,1,12.9
3,1.661683,3,5.3
4,4.475450,5,16.0


In [129]:
uber_df.loc[:, 'distance'] = round(uber_df['distance'])
uber_df.loc[:, 'fare_amount'] = round(uber_df['fare_amount'])
uber_df.head()

,distance,passenger_count,fare_amount
0,2.0,1,8.0
1,2.0,1,8.0
2,5.0,1,13.0
3,2.0,3,5.0
4,4.0,5,16.0


In [130]:
"""
titanic_df.loc[:,'pickup_longitude']=titanic_df['Pclass'].fillna(titanic_df['Pclass'].mode()).astype(int)
titanic_df.loc[:,'pickup_latitude']=titanic_df['Age'].fillna(titanic_df['Age'].median())
titanic_df.loc[:,'Age']=(titanic_df['Age']/10).astype(str).str[0].astype(int)*10
titranic_df = titanic_df.dropna()
X = titanic_df.drop("Survived",axis=1)
y = titanic_df["Survived"]
#probs = Lab2_helper.class_conditional(X,y)
#priors = Lab2_helper.compute_priors(y)
x = titanic_df.drop("Survived",axis=1).loc[2]
#post_probs = Lab2_helper.posteriors(probs,priors,x)
"""

'\ntitanic_df.loc[:,\'pickup_longitude\']=titanic_df[\'Pclass\'].fillna(titanic_df[\'Pclass\'].mode()).astype(int)\ntitanic_df.loc[:,\'pickup_latitude\']=titanic_df[\'Age\'].fillna(titanic_df[\'Age\'].median())\ntitanic_df.loc[:,\'Age\']=(titanic_df[\'Age\']/10).astype(str).str[0].astype(int)*10\ntitranic_df = titanic_df.dropna()\nX = titanic_df.drop("Survived",axis=1)\ny = titanic_df["Survived"]\n#probs = Lab2_helper.class_conditional(X,y)\n#priors = Lab2_helper.compute_priors(y)\nx = titanic_df.drop("Survived",axis=1).loc[2]\n#post_probs = Lab2_helper.posteriors(probs,priors,x)\n'

Defining the Gaussian Bayesian Classifier

In [140]:
class GaussianBayes:
    
    def compute_priors(self, y):
        value_counts = y.value_counts().sort_index()
        total_values = len(y)
        indexes = [y.name+"="+str(v) for v in value_counts.index.tolist()]
        priors = dict(zip(indexes, value_counts / total_values))
        return priors
    
    def compute_normal_distribution(self, mean, stddev, x):
        exp = (((x-mean) / stddev)**2) * -0.5
        denom = ((stddev) * math.sqrt(2 * math.pi))
        return (math.e ** exp) / denom
    
    
    # Since we are assuming a normal distribution, this is where we use the mean, std, and variance to calculate the class conditional probability. 
    def specific_class_conditional(self,x,xv,y,yv):
        # todo: replace binning with normal distribution calculations
        likelihoods = {}
        priors = {}
        for ux in x.unique():
            y_vals = y.loc[x==ux]
            for uy in y_vals.unique():
                prob = self.compute_normal_distribution(y_vals.mean(), y_vals.std(), uy)
                likelihoods[ux] = prob
                priors[ux] = len(y_vals) / y
        denom = 0
        
        for k in priors.keys():
            denom += (priors[k]*likelihoods[k])

        print(likelihoods, priors)
        classcond = likelihoods[xv]*priors[xv] / denom
        return classcond


    def class_conditional(self,X,y):
        probs = {}
        
        for eachy in y.unique():
            for col in X.columns:
                for eachx in X[col].unique():
                    probs[col + "=" + str(eachx) + "|" + y.name + "=" + str(eachy)] = specific_class_conditional(X[col], eachx, y, eachy)
        return probs

    def posteriors(self,probs,priors,x):
        post_probs = {}
        denom = 0
        for k in priors.keys():
            numerator = 1
            postkey = ""
            
            for idx in x.index:
                postkey += idx + "=" + str(x[idx]) + ","
                probkey = idx + "=" + str(x[idx]) + "|" + k
                
                if probkey not in probs:
                    numerator *= 0
                else:
                    numerator *= probs[probkey]
            
            numerator*=priors[k]
            post_probs[k + "|" + postkey[:-1]] = numerator
            denom+=numerator
            
        for k, v in post_probs.items():
            if denom != 0:
                post_probs[k] = v / denom
            else:
                post_probs[k] = 1 / len(list(priors.keys()))
        
        return post_probs


In [141]:
gb = GaussianBayes()

In [142]:
prob = gb.specific_class_conditional(uber_df['distance'], 2, uber_df['fare_amount'], 5)
prob

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


{2.0: 8.181811585777186e-36, 5.0: 1.8984312096384113e-18, 4.0: 5.404057464812982e-13, 0.0: 8.155841010743764e-11, 12.0: 8.198942528874093e-06, 1.0: 3.1879095350432375e-22, 3.0: 1.4320009404007173e-55, 6.0: 1.5052117381184512e-08, 13.0: 0.0019454663398786216, 11.0: 6.402513266824265e-05, 10.0: 0.003744655660433535, 8.0: 4.584254257140247e-06, 16.0: 1.4245407350483897e-05, 7.0: 9.308843616662131e-05, 9.0: 7.996877777356592e-05, 14.0: 5.9562237355852044e-05, 20.0: 0.02745481972268529, 18.0: 2.053264228085836e-07, 19.0: 0.0007003648431795029, 17.0: 0.002095195070784726, 8666.0: 0.016175985809465405, 22.0: 3.861276497488424e-07, 15.0: 0.0001941056289284621, 21.0: 0.048143129661542515, 33.0: 0.01895681920415256, 8647.0: 0.004630715083753595, 8667.0: 0.018930345377931024, 23.0: 0.00018108010205114978, 6021.0: 0.10984782236693061, 5941.0: 0.018307970394488435, 8665.0: 0.00033598974755914043, 8654.0: nan, 6032.0: 0.09317639016847351, 4530.0: nan, 8648.0: 0.008301535751621614, 3829.0: nan, 8664.

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
199995   NaN
199996   NaN
199997   NaN
199998   NaN
199999   NaN
Name: fare_amount, Length: 199999, dtype: float64